In [1]:
import os
os.environ['JAVA_HOME'] = '/dk/jdk/jdk-21.0.3'
os.environ['PYSERINI_CACHE'] = '/dk/rag-dataset/'
io_path = '/dk/lcj_rag/pyserini-autofusion/experiments/runs'

depth=1000
#  alpha*bf + (1-alpha)*bm25
alpha = 0.5
nq = 1024
k=100

bf_trec_path = f'{io_path}/run.dpr.trivia-test.bm25.trec'
bm25_trec_path = f'{io_path}/run.dpr.trivia-test.multi.bf.trec'
fusion_tag = 'fusion'
out_trec_path = f'{io_path}/run.dpr.trivia-test.multi.bf.bm25.{fusion_tag}.alpha-{alpha}.nq-{nq}.k-{k}.trec'
out_json_path = f'{io_path}/run.dpr.trivia-test.multi.bf.bm25.{fusion_tag}.alpha-{alpha}.nq-{nq}.k-{k}.json'

In [2]:
cmd0 = f'python -m pyserini.autofusion \
        --runs {bf_trec_path} {bm25_trec_path} \
        --runtag {fusion_tag} \
        --method interpolation  \
        --alpha {alpha} \
        --depth {depth}  \
        --nq {nq}  \
        --k {k} \
        --output {out_trec_path}'
# print(cmd0)
os.system(cmd0)

0

In [3]:
cmd1 = f'python -m pyserini.eval.convert_trec_run_to_dpr_retrieval_run \
  --index wikipedia-dpr-100w \
  --topics dpr-trivia-test \
  --input {out_trec_path} \
  --output {out_json_path}'
os.system(cmd1)

checking path: /dk/rag-dataset/indexes/lucene-index.wikipedia-dpr-100w.20210120.d1b9e6...


Apr 26, 2024 2:58:29 PM org.apache.lucene.store.MemorySegmentIndexInputProvider <init>
INFO: Using MemorySegmentIndexInput with Java 21; to disable start with -Dorg.apache.lucene.store.MMapDirectory.enableMemorySegments=false
100%|██████████| 102400/102400 [02:19<00:00, 735.49it/s]


0

In [4]:
cmd2 = f'python -m pyserini.eval.evaluate_dpr_retrieval \
  --retrieval {out_json_path} \
  --topk 100'
filename = f'{io_path}/run.dpr.trivia-test.multi.bf.bm25.{fusion_tag}.alpha-{alpha}.nq-{nq}.k-{k}.res'
res = os.system(cmd2 + ' 1>' + filename)


100%|██████████| 1024/1024 [00:00<00:00, 27443.18it/s]


In [28]:
import numpy as np
from tqdm import tqdm

import os
os.environ['JAVA_HOME'] = '/dk/jdk/jdk-21.0.3'
os.environ['PYSERINI_CACHE'] = '/dk/rag-dataset/'
io_path = '/dk/lcj_rag/pyserini-autofusion/experiments/runs'

depth=1000
#  alpha*bf + (1-alpha)*bm25
alpha = 0.5
nq = -1
k=100

bf_trec_path = f'{io_path}/run.dpr.trivia-test.bm25.trec'
bm25_trec_path = f'{io_path}/run.dpr.trivia-test.multi.bf.trec'
fusion_tag = 'fusion'

for alpha in tqdm(np.arange(0, 1.05, 0.05)):
  alpha = round(alpha, 2)
  out_trec_path = f'{io_path}/run.dpr.trivia-test.multi.bf.bm25.{fusion_tag}.alpha-{alpha}.nq-{nq}.k-{k}.trec'
  out_json_path = f'{io_path}/run.dpr.trivia-test.multi.bf.bm25.{fusion_tag}.alpha-{alpha}.nq-{nq}.k-{k}.json'
  cmd0 = f'python -m pyserini.autofusion \
          --runs {bf_trec_path} {bm25_trec_path} \
          --runtag {fusion_tag} \
          --method interpolation  \
          --alpha {alpha} \
          --depth {depth}  \
          --nq {nq}  \
          --k {k} \
          --output {out_trec_path}'
  # print(cmd0)
  os.system(cmd0)

  cmd1 = f'python -m pyserini.eval.convert_trec_run_to_dpr_retrieval_run \
    --index wikipedia-dpr-100w \
    --topics dpr-trivia-test \
    --input {out_trec_path} \
    --output {out_json_path}'
  os.system(cmd1)

  cmd2 = f'python -m pyserini.eval.evaluate_dpr_retrieval \
    --retrieval {out_json_path} \
    --topk 100'
  filename = f'{io_path}/run.dpr.trivia-test.multi.bf.bm25.{fusion_tag}.alpha-{alpha}.nq-{nq}.k-{k}.res'
  res = os.system(cmd2 + ' 1>' + filename)


  0%|          | 0/21 [00:00<?, ?it/s]

In [26]:
from collections import defaultdict

set_list = []

alpha_fail_list = defaultdict(list)
num_fail_alpha = defaultdict(list)

for alpha in tqdm(np.arange(0, 1.05, 0.05)):
  alpha = round(alpha, 2)
  # alpha = f'{alpha:.2f}'
  print(alpha)
  filename = f'{io_path}/run.dpr.trivia-test.multi.bf.bm25.{fusion_tag}.alpha-{alpha}.nq-{nq}.k-{k}.res'
  s = set()
  with open(filename, 'r') as file:
    line = file.readline()
    while line:
      s.add(line)
      line = file.readline()
  set_list.append(s)
  alpha_fail_list[alpha].append(len(s))
  num_fail_alpha[len(s)].append(alpha)

for i in range(1, len(set_list)):
  set_list[i] &= set_list[i - 1]

for fail in sorted(set_list[-1]):
  print(fail)
print(f'number of hard fails: {len(set_list[-1])}')
print(f'acc upbound: {(nq - len(set_list[-1])) / nq}')

for (num_fail, alpha) in num_fail_alpha.items():
  print(num_fail, (nq - num_fail) / nq, alpha)

100%|██████████| 21/21 [00:00<00:00, 7523.74it/s]

0.0
0.05
0.1
0.15
0.2
0.25
0.3
0.35
0.4
0.45
0.5
0.55
0.6
0.65
0.7
0.75
0.8
0.85
0.9
0.95
1.0
qid: 10	Top100	has_answer: 0

qid: 106	Top100	has_answer: 0

qid: 112	Top100	has_answer: 0

qid: 119	Top100	has_answer: 0

qid: 130	Top100	has_answer: 0

qid: 149	Top100	has_answer: 0

qid: 173	Top100	has_answer: 0

qid: 212	Top100	has_answer: 0

qid: 221	Top100	has_answer: 0

qid: 224	Top100	has_answer: 0

qid: 229	Top100	has_answer: 0

qid: 233	Top100	has_answer: 0

qid: 240	Top100	has_answer: 0

qid: 242	Top100	has_answer: 0

qid: 245	Top100	has_answer: 0

qid: 253	Top100	has_answer: 0

qid: 38	Top100	has_answer: 0

qid: 44	Top100	has_answer: 0

qid: 48	Top100	has_answer: 0

qid: 53	Top100	has_answer: 0

qid: 7	Top100	has_answer: 0

qid: 70	Top100	has_answer: 0

qid: 87	Top100	has_answer: 0

qid: 9	Top100	has_answer: 0

number of hard fails: 24
acc upbound: 0.90625
37 0.85546875 [0.0, 0.95]
35 0.86328125 [0.05, 0.9]
33 0.87109375 [0.1]
31 0.87890625 [0.15]
30 0.8828125 [0.2]
28 0.890625 [0.